In [1]:
import os
import pandas as pd
import geopandas as gpd
import folium
import itertools
from pathlib import Path

# Completar con el directorio donde se encuentra clonado e instalado el repositorio de UrbanTrips
URBANTRIPS_PATH = "RUTA/DE/URBANTRIPS"
URBANTRIPS_PATH = Path(URBANTRIPS_PATH)
os.chdir(URBANTRIPS_PATH)

from urbantrips.utils import utils
from urbantrips.kpi import overlapping as ovl
from urbantrips.viz import overlapping as ovl_viz


# Leer archivos de configuración y conexiones a las db
configs = utils.leer_configs_generales()
alias = configs['alias_db_data']
conn_data = utils.iniciar_conexion_db(tipo='data')
conn_insumos = utils.iniciar_conexion_db(tipo='insumos')

In [ ]:
# Se leen los datos de las lineas
metadata_lineas = pd.read_sql("select id_linea,nombre_linea, modo from metadata_lineas;", conn_insumos)
# Se puede buscar por nombre de linea que contenga alguna palabra o numero
metadata_lineas[metadata_lineas.nombre_linea.str.contains("50") #reemplazar 50 por lo que se desee buscar en el nombre de la linea
                ]

In [3]:
# Configurar el día a procesar 
day = "weekday"

# La resolucion h3 (no puede ser mayor a la que aparece en las configuraciones)
h3_res_comp = 8

# Los id de las lineas a comparar
comp_line_id = 1
base_line_id = 2

In [4]:
# Obtiene del archivo de configuración si se deben usar ramales o lineas
use_branches = configs['lineas_contienen_ramales']
if use_branches:
    # Lee los datos de los ramales
    metadata = pd.read_sql(f"select id_linea,id_ramal from metadata_ramales where id_linea in ({base_line_id},{comp_line_id})",
                            conn_insumos,dtype={'id_linea': int, 'id_ramal': int})
    route_type = 'branches'

    # Computa todas las posibles combinaciones de ramales entre esas dos lineas
    route_id_combinations = list(itertools.combinations(metadata['id_ramal'], 2))
    base_route_id_combinations = list(itertools.combinations(metadata.loc[metadata.id_linea == base_line_id,'id_ramal'], 2))
    comp_line_id_combinations = list(itertools.combinations(metadata.loc[metadata.id_linea == comp_line_id,'id_ramal'], 2))
    route_id_combinations = [combination for combination in route_id_combinations if ((combination not in base_route_id_combinations) and (combination not in comp_line_id_combinations))]
    
    metadata_branches = pd.read_sql(f"select * from metadata_ramales where id_linea in ({base_line_id},{comp_line_id})",
                                conn_insumos,dtype={'id_linea': int, 'id_ramal': int})

else:
    route_type = 'lines'
    route_id_combinations = [(base_line_id, comp_line_id)]



In [ ]:
# Aca se pueden ver todos los ramales de las lineas a comparar
metadata_branches   

In [ ]:
# Selecciona un par de las posibles combinaciones de ramales
route_id_combination = route_id_combinations[0] 
route_id_combination

In [ ]:
# crea un id de ruta unico de ramal o linea en funcion de si esta configurado para usar ramales o lineas
if use_branches:
    if route_id_combination[0] in metadata.loc[metadata.id_linea == base_line_id,'id_ramal'].values:
        base_route_id = route_id_combination[0]
        comp_route_id = route_id_combination[1]

    else:
        base_route_id = route_id_combination[1]
        comp_route_id = route_id_combination[0]
    
    nombre_ramal_base = metadata_branches.loc[metadata_branches.id_ramal == base_route_id,'nombre_ramal'].item()
    nombre_ramal_comp = metadata_branches.loc[metadata_branches.id_ramal == comp_route_id,'nombre_ramal'].item()

    base_route_str = f"ramal {nombre_ramal_base} (id {base_route_id})"
    comp_route_str = f"ramal {nombre_ramal_comp} (id {comp_route_id})"

else:
    base_route_id,comp_route_id = route_id_combination
    base_route_str = ""
    comp_route_str = ""

nombre_linea_base = metadata_lineas.loc[metadata_lineas.id_linea == base_line_id,'nombre_linea'].item()
nombre_linea_comp = metadata_lineas.loc[metadata_lineas.id_linea == comp_line_id,'nombre_linea'].item()

print(f"Tomando como linea base la linea {nombre_linea_base} (id {base_line_id}) " + base_route_str)
print(f"Tomando como linea comparacion la linea {nombre_linea_comp} (id {comp_line_id}) " + comp_route_str)



In [ ]:
# Calcula la superposicion de la oferta de la linea base con la de la linea de comparacion
overlapping_dict = ovl.compute_supply_overlapping(day, base_route_id,comp_route_id,route_type,h3_res_comp)  
base_gdf = overlapping_dict["base"]["h3"]
base_route_gdf = overlapping_dict["base"]["line"]
comp_gdf = overlapping_dict["comp"]["h3"]
comp_route_gdf = overlapping_dict["comp"]["line"]


In [ ]:
# Se visualiza la superposicion de la oferta de la linea base con la de la linea de comparacion
f = ovl_viz.plot_interactive_supply_overlapping(overlapping_dict)
f.save(f"resultados/html/{alias}_supply_overlapping_base_{base_route_id}_comp_{comp_route_id}_h3_{h3_res_comp}.html")
f

In [ ]:
# Calcula la demanda de la linea base y la de comparacion
base_demand,comp_demand = ovl.compute_demand_overlapping(base_line_id,comp_line_id,day,
                                                         base_route_id,comp_route_id,
                                                         base_gdf,comp_gdf)

In [ ]:
# Se visualiza la superposicion de la demanda de la linea base con la de la linea de comparacion
fig = ovl_viz.plot_interactive_demand_overlapping(base_demand, comp_demand, overlapping_dict)
fig.save(f"resultados/html/{alias}_demand_overlapping_base_{base_route_id}_comp_{comp_route_id}_h3_{h3_res_comp}.html")
fig